# Comparing BERT Models

In [ ]:
!pip install toxy-bot==0.1.30

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 726.7/726.7 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.4/219.4 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.3/443.3 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import comet_ml

import torch
from google.colab import userdata

import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import CometLogger

import os

from toxy_bot.ml.datamodule import AutoTokenizerDataModule
from toxy_bot.ml.module import SequenceClassificationModule
from toxy_bot.ml.utils import create_dirs, create_experiment_name
from toxy_bot.ml.config import Config, DataModuleConfig, ModuleConfig, TrainerConfig

First, let's configure some basic settings

In [ ]:
# Model and dataset
lr = ModuleConfig.learning_rate

dataset_name = DataModuleConfig.dataset_name
train_size = DataModuleConfig.train_size
train_split = DataModuleConfig.train_split
batch_size = DataModuleConfig.batch_size
max_length = DataModuleConfig.max_length

print(f"Learning rate: {lr}")
print()
print(f"Dataset: {dataset_name}")
print(f"Train size: {train_size}")
print(f"Batch size: {batch_size}")
print(f"Max length: {max_length}")

Learning rate: 3e-05

Dataset: anitamaxvim/jigsaw-toxic-comments
Train size: 0.85
Batch size: 64
Max length: 256


In [ ]:
# Paths
this_nb = os.getcwd()

cache_dir = os.path.join(this_nb, "data")
log_dir = os.path.join(this_nb, "logs")
ckpt_dir = os.path.join(this_nb, "checkpoints")
perf_dir = os.path.join(log_dir, "perf")

print(f"Cache dir: {cache_dir}")
print(f"Log dir: {log_dir}")
print(f"Checkpoint dir: {ckpt_dir}")
print(f"Perf dir: {perf_dir}")

torch.set_float32_matmul_precision(precision="medium")

# Create required directories
create_dirs([cache_dir, log_dir, ckpt_dir, perf_dir])

Cache dir: /content/data
Log dir: /content/logs
Checkpoint dir: /content/checkpoints
Perf dir: /content/logs/perf


In [ ]:
# Validation and logging
check_val_every_n_epoch = TrainerConfig.check_val_every_n_epoch
val_check_interval = TrainerConfig.val_check_interval
log_every_n_steps = TrainerConfig.log_every_n_steps

print(f"Check val every n epoch: {check_val_every_n_epoch}")
print(f"Val check interval: {val_check_interval}")
print(f"Log every n steps: {log_every_n_steps}")

Check val every n epoch: 1
Val check interval: None
Log every n steps: 200


Now, we can define our LightningDataModule, which will be used by Trainer for its DataLoaders.

In [ ]:
models = [
    "google/bert_uncased_L-2_H-128_A-2", # BertTiny (4M)
    "google/bert_uncased_L-8_H-512_A-8", # BertMedium (42M)
    "google/bert_uncased_L-12_H-768_A-12", # BertBase (108M)
]

for model in models:

    lit_datamodule = AutoTokenizerDataModule(
        model_name=model,
        dataset_name=dataset_name,
        cache_dir=cache_dir,
        batch_size=batch_size,
        max_length=max_length,
        train_split=train_split,
        train_size=train_size,
    )

    lit_model = SequenceClassificationModule(
        model_name=model,
        learning_rate=lr,
    )

    callbacks = [
        ModelCheckpoint(dirpath=ckpt_dir, filename=model),
    ]

    comet_logger = CometLogger(
        api_key=userdata.get("COMET_API_KEY"),
        project_name="toxy-bot-compare-models",
        workspace="anitamaxvim",
        name=model,
    )

    lit_trainer = pl.Trainer(
        accelerator="auto",
        devices="auto",
        strategy="auto",
        precision="16-mixed",
        max_epochs=5,
        logger=comet_logger,
        callbacks=callbacks,
        val_check_interval=val_check_interval,
        log_every_n_steps=log_every_n_steps,
        check_val_every_n_epoch=check_val_every_n_epoch,
    )

    lit_trainer.fit(model=lit_model, datamodule=lit_datamodule)
    comet_logger.experiment.end()

config.json:   0%|          | 0.00/382 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/382 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

...

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name                | Type                          | Params | Mode 
------------------------------------------------------------------------------
0 | model               | BertForSequenceClassification | 109 M  | eval 
1 | accuracy            | MultilabelAccuracy            | 0      | train
2 | f1_score            | MultilabelF1Score             | 0      | train
3 | precision           | MultilabelPrecision           | 0      | train
4 | recall              | MultilabelRecall              | 0      | train
5 | macro_avg_accuracy  | MultilabelAccuracy            | 0      | train
6 | macro_avg_f1_score  | MultilabelF1Score             | 0      | train
7 | macro_avg_precision | MultilabelPrecision           | 0      | train
8 | macro_avg_recall    | MultilabelRecall              | 0      | train
-----------------------------------------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
# models = [
    # "google/bert_uncased_L-2_H-128_A-2" # BertTiny (4M)
    # "google/bert_uncased_L-4_H-256_A-4", # BertMini (?)
    # "google/bert_uncased_L-4_H-512_A-8", # BertSmall (29M)
    # "google/bert_uncased_L-8_H-512_A-8", # BertMedium (42M)
    # "google/bert_uncased_L-12_H-768_A-12", # BertBase (108M)
# ]
